# GADM table format optimizer

- We received a country from a client that wants us to import to the system. In order to acoomplish that,
we have to make space for that country on the system. That already happens in "GADM-processing-layers".
- Here we will addapt the layer country fields
- Process:
    - Read the gadm file and get the column structure
    - Identify fromt he client country the fields that are usefull
    - Modifiy the table structure

1. toma el valor de GID
2. añadele a GID el valor que se ha ido contando sobre el

UNFINISHED

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
"""Set the inputs"""
gadm_file= r"C:\Users\ruben.crespo\Documents\03_tests\administrative_units\00_gadm\gadm404-levels.gpkg"
gadm_level1_gdf = gpd.read_file(gadm_file, layer="level1")
client_file = r"C:\Users\ruben.crespo\Documents\03_tests\administrative_units\09_norway_case\fylker_2022_wgs84_navn.shp"
client_gdf = gpd.read_file(client_file)

- This is the column structure we are following. ['CID', 'Shape_Leng', 'Shape_Area', 'Country_n', 'GID', 'GID_list', 'GID_n', 'geometry']
- We will create the table by columns
- Depending of the the sublevel, we have to rethink in another way

In [3]:
"""Create the CID column"""
CID_id = 'NOR'
CID_s = pd.Series(CID_id, index=client_gdf.index)
CID_s.name = 'CID'
client_gadm_df = pd.DataFrame(CID_s)

#{'Gene':s.index, 'count':s.values}

In [4]:
"""Create the Country_n column"""
Country_name = 'Norway'
Country_name_s = pd.Series(Country_name, index=client_gdf.index)
Country_name_s.name = 'Country_n'
client_gadm_df = pd.concat([client_gadm_df, Country_name_s], axis=1) #concat series to dataframe

In [8]:
"""Define the target and the base"""

country_gdf = gadm_level1_gdf.loc[(gadm_level1_gdf["ID_0"] == CID_id)]


if len(country_gdf) > len(gadm_level1_gdf):
    base_gdf = client_gdf
    target_gdf = country_gdf
else:
    base_gdf = country_gdf
    target_gdf = client_gdf

print("client_gdf lenght is: ", len(client_gdf), "and the gadm lenght is: ", len(country_gdf))

client_gdf lenght is:  11 and the gadm lenght is:  19


In [19]:
"""We create the table with the polygon centres"""
base_centroids_gdf = base_gdf.set_geometry(base_gdf['geometry'].centroid)

C:\Users\ruben.crespo\AppData\Local\Temp\ipykernel_2300\805165818.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  base_centroids_gdf = base_gdf.set_geometry(base_gdf['geometry'].centroid)


In [25]:
for index_base, row_base in base_centroids_gdf.iterrows(): #row is a series
    # print(row_base['geometry'])
    for index_target, row_target in target_gdf.iterrows():
        print(row_base['geometry'].intersects(row_target['geometry']))

False
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
False
Fa

In [ ]:
"""Time for GID (country CID with code)"""
"""
we have to make it match with the gadm layer, it might not match. To locate who goes to ho
we use central point and catch the polygons
two codes, one for the region, and other for the subregions, both look at the corresponding gadm levels 
"""


In [24]:
client_gadm_df.head()

,CID,Country_n
0,NOR,Norway
1,NOR,Norway
2,NOR,Norway
3,NOR,Norway
4,NOR,Norway


In [7]:
"""Make a relation with the fields of the country with the gadm layer"""

client_gdf.head()

,OBJECTID,objid,objtype,kommunenum,samiskforv,lokalid,navnerom,versjonid,datafangst,oppdaterin,...,opphav,SHAPE_Leng,SHAPE_Area,OBJECTID_1,objtype_1,navn,rekkefolge,sprak,kommune_fk,geometry
0,1,1,Kommune,5031,0,fe328f8d-fbde-4c21-ab28-cbc583be401f,http://skjema.geonorge.no/SOSI/produktspesifik...,4.1,None,2020-02-07,...,None,2413.797439,3.732041e+05,1,Kommune,Malvik,0,nor,1,"POLYGON ((10.85543 63.34641, 10.85535 63.34684..."
1,2,2,Kommune,4636,0,dadfaa2a-7413-4fb4-a6f5-aa3b5dbf0507,http://skjema.geonorge.no/SOSI/produktspesifik...,4.1,None,2020-02-07,...,None,174625.034566,1.759958e+09,2,Kommune,Solund,0,nor,2,"POLYGON ((4.10297 61.22995, 4.37810 61.22315, ..."
2,3,3,Kommune,3019,0,5b6fe7ff-260e-4a9e-b203-b83a71eb0a56,http://skjema.geonorge.no/SOSI/produktspesifik...,4.1,None,2020-02-07,...,None,71857.553659,1.515626e+08,3,Kommune,Vestby,0,nor,3,"POLYGON ((10.63304 59.57378, 10.63315 59.57466..."
3,4,4,Kommune,3032,0,ffd39f1e-f20e-4559-80c9-42b911aac29b,http://skjema.geonorge.no/SOSI/produktspesifik...,4.1,None,2020-02-07,...,None,50923.680978,8.322558e+07,4,Kommune,Gjerdrum,0,nor,4,"POLYGON ((10.93197 60.08286, 10.93168 60.08329..."
4,5,5,Kommune,3043,0,e4f877ca-9330-4830-85d8-d116a738be2a,http://skjema.geonorge.no/SOSI/produktspesifik...,4.1,None,2000-06-30,...,None,214158.193080,1.177789e+09,5,Kommune,Ål,0,nor,5,"POLYGON ((8.25546 60.69480, 8.25565 60.69587, ..."


How to concatenate data: https://datatofish.com/concatenate-values-python/

**Note for myself.** There is no way to concatenate a str and an int together

In [15]:
# add test column and concatenate the GID with an acumulative number grouped by the GID value
gdf["test"] = gdf['GID'] + (gdf.groupby('GID').cumcount()+1).map(str)
gdf["test"]


0      ABW1
1      AFG1
2      AGO1
3      AIA1
4      ALA1
       ... 
278    CHL3
279    CHL4
280    FJI1
281    FJI2
282    FJI3
Name: test, Length: 283, dtype: object

In [7]:

gdf[["test", "GID_unik","GID"]]
gdf.dtypes

Shape_Area     float64
Country_n       object
GID_unik        object
fd              object
GID             object
geometry      geometry
test            object
dtype: object

In [16]:

#gdf['test_list'] = gdf.groupby('GID')['test'].apply(list)
#https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings
gdf['test_list'] = gdf.groupby('GID')['test'].apply(lambda x: x.sum())
gdf['test_list']
#gdf[["test", "test_list"]]


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
278    NaN
279    NaN
280    NaN
281    NaN
282    NaN
Name: test_list, Length: 283, dtype: object

In [11]:
df = pd.DataFrame({'A': 'a a b'.split(), 'B': [1,2,3], 'C': [4,6, 5]})
g = df.groupby('A')
g.apply(lambda x: "{%s}" %  ', '.join(x))

A
a    {A, B, C}
b    {A, B, C}
dtype: object

In [ ]:
def func(row):
    if row['mobile'] == 'mobile':
        return 'mobile'
    elif row['tablet'] =='tablet':
        return 'tablet' 
    else:
        return 'other'

In [ ]:


for index, content in gdf.iterrows():   

    # get 'not disjoint' countries
    neighbors = gdf[~gdf.geometry.disjoint(content.geometry)].NAME.tolist()

    # remove own name of the country from the list
    neighbors = [ name for name in neighbors if content.NAME != name ]

    # add names of neighbors as NEIGHBORS value
    gdf.at[index, "NEIGHBORS"] = ", ".join(neighbors)
   
# save GeoDataFrame as a new file
gdf.to_file("c:/path/to/newfile.shp")